# DREAMFUSION: TEXT-TO-3D USING 2D DIFFUSION

참고링크
+ blogs
    + https://jaehoon-daddy.tistory.com/92
    + https://kimjy99.github.io/%EB%85%BC%EB%AC%B8%EB%A6%AC%EB%B7%B0/dreamfusion/
+ paper : https://arxiv.org/abs/2209.14988


## Contribution
+ diffusion 모델을 이용해 다양한 angle에서의 2D image를 생성해내 prior로 하여 이를 재구성하여 3D 이미지를 생성해낸다.

## 기존 방법론
1. 3D이미지를 그대로 학습하고 그대로 생성
2. CLIP + NeRF 하여 생성(Dream fields)

=> 많은 데이터 셋이 필요하거나, 낮은 reality를 가졌다..

2번의 방법론에서 CLIP에 해당하는 부분에 diffusion모델을 사용해보겠다.

## Method

<img src="https://drive.google.com/uc?id=1Zqj-lG8tkae4ytCtzUDS7YNwPPSL0Jod">

__방법__  
1. NeRF-Like 모델을 random initialize한다.
2. 이를 통하여 random한 angle의 view를 생성한다.(처음엔 학습이 덜되어서 이상할것임)
3. 해당 view에 noise를 넣고, Diffusion 모델을 통해 텍스트 프롬프팅으로 fidelity를 추가해준다.
4. 이 때 time step에 해당하는 score로부터 NeRF파라미터의 gradient를 계산한다.
5. NeRF 파라미터 갱신

### Score Distailation Sampling(SDS)



기존의 Diffusion loss는 다음과 같이 주어진다.

$\mathcal{L}_{Diff}(\phi, \mathbf{x}) = \mathbb{E}_{t \sim u(0, 1), \epsilon \sim \mathcal{N}(0, I)}\big[w(t) ||\epsilon_\phi(\alpha_t\mathbf{x} + \sigma_t\epsilon ; t) -\epsilon||_2^2 \big]$

이제, datapoint를 생성해내기 위해서 원래는 pixel space에서 바로 샘플링했지만, ($\mathbf{x}$) 이를 파라미터 space인 $\theta$를 이용해 샘플링하는 것을 제안

+ $\mathbf{x} = g(\theta)$ : 변환된 파라미터로 생성된 이미지
+ $g$ : differentiable generator (flip, rotate등의 변환)
+ $\theta$ : NeRF-Like 모델을 통해 랜더링 된 이미지(변환 과정을 거칠 때, 미분 가능한 계산을 이용하고 그 계산에 필요한 파라미터를 의미함 : MLP)

$$\theta^* = \arg \min_\theta \mathcal{L}_{diff}(\phi, \mathbf{x} = g(\theta))$$

즉, 위의 optimal한 $\theta$를 구해내면, 우리가 원하는 이미지 변환(sampling)을 파라미터를 통해 진행할 수 있게된다.  
그리고 이 때의 $\mathbf{x}$를 generated datapoint라고 이야기 한다.

따라서, 이제 해야 할 일은 어떻게 loss로부터 $\theta$까지 gradient를 흘려보낼 것인가가 중요한 문제이다.

해당 paper에서는 condition diffusion에 연쇄법칙을 이용하여 loss계산을 한다.

$f$를 목적함수라고하면,   
$\nabla_\theta f = \cfrac{\partial{f}}{\partial{\hat{\epsilon}}}\cfrac{\partial{\hat{\epsilon}}}{\partial{z}}\cfrac{\partial{z}}{\partial{x}}\cfrac{\partial{x}}{\partial{\theta}}$ 이와 같은 연쇄법칙을 계산하면 다음의 식이 나온다.

$w(t)(\hat{\epsilon}_\theta(z_t ; y, t)-\epsilon)\cfrac{\partial{\hat{\epsilon}(z_t ; y, t)}}{\partial{z_t}} \cfrac{\partial{z_t}}{\partial{\mathbf{x}}}\cfrac{\partial{\mathbf{x}}}{\partial{\theta}}$

여기에 필요없는 상수 term은 $w(t)$로 흡수시키고, $\cfrac{\partial{\hat{\epsilon}(z_t ; y, t)}}{\partial{z_t}}$는 연산량이 너무 많을 뿐더러, 너무 작은 noise 수준에서의 컨트롤이어서 생략한다.

$\nabla_\theta\mathcal{L}_{SDS}(\phi, \mathbf{x} = g(\theta)) \simeq \mathbb{E}_{t, \epsilon} \bigg[w(t)(\hat{\epsilon}_\phi(z_t ; y, t) - \epsilon) \cfrac{\partial{\mathbf{x}}}{\partial{\theta}} \bigg]$

## Rendering Algorithm

NeRF의 idea는 입력이미지와 위치정보가 주어져서 그 위치에서 바라보았을 때의 view와 ground truth 사이의 loss를 계산하여 최적화하고 이를 통하여 보지 못한 angle에서의 이미지를 생성시키는 방식이다.

해당 논문에선 이와 더불어 light 정보를 추가하여 그림자를 포함하여 생성한다.

### Shading

빛을 통해 반사된 RGB 알베도 값을 $\rho$라고 할 때, 다음과 같이 각 포인트를 표현한다.  
$$(\tau, \rho) = MLP(\mu ; \theta)$$


+ $\tau$ : 볼륨 density
+ $\rho$ : RGB 알베도 (물체의 근본적인 표면 색상)
+ $\mu$ : 3D 좌표
+ $\theta$ : rendering된 이미지

이 정보를 통해 생성되는 이미지($\mathbf{c}$)는 다음과 같이 표현한다.
$$\mathbf{c} = \rho \circ(\mathcal{\ell_\rho}\circ \max(0, \mathbf{n}\cdot(\ell - \mu) / ||\ell - \mu||) + \ell_\alpha)$$

+ $\mathbf{n} = -\nabla_\mu \tau / ||\nabla_\mu \tau||$ : normalize된 density의 negative gradient
+ $\ell$ : 예측하고자하는 좌표점
+ $\ell_\rho$ : 해당 좌표점의 color
+ $\ell_\alpha$ : 해당 좌표점의 주변 color

<img src="https://drive.google.com/uc?id=1DOXo8gSBIQBMghNi6l680QPPIX6GnTMq" height=300>